In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def gs_crit_full(df):
    overfit = np.abs(df.best_train_auc-df.best_val_auc) + np.abs(df.best_train_auc-df.best_test_auc) + \
              np.abs(df.best_train_auc-df.best_test_intra_auc)
    best_perf = 1-df.best_train_auc + 1-df.best_val_auc + 1-df.best_test_intra_auc + \
                1-df.best_test_auc
    
    return overfit + 2*best_perf

# S.or.

In [7]:
bdd_sor = pd.read_csv("/neurospin/dico/agaudin/Runs/09_new_repo/Output/grid_searches/step2/S.or./bdd_Sor_both.csv", index_col=0)

bdd_sor['best_crit'] = gs_crit_full(bdd_sor)
bdd_sor['min_best_perf'] = bdd_sor[['best_train_auc', 'best_val_auc', 'best_test_intra_auc', 'best_test_auc']].apply(min, axis=1)
bdd_sor['min_best_intra'] = bdd_sor[['best_train_auc', 'best_val_auc', 'best_test_intra_auc']].apply(min, axis=1)

# sort by criterion
bdd_sor.sort_values(by='best_crit', inplace=True)

bdd_sor

,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,best_crit,min_best_perf,min_best_intra
30,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.451216,0.523493,0.518121,0.470443,0.539439,0.604158,0.560254,0.536022,0.035590,0.085306,1,0.036188,1.941209,3.609206,0.536022,0.536022
35,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.904493,0.498545,0.506342,0.667796,0.857813,0.633680,0.480973,0.624384,0.010431,0.003729,3,0.007851,1.079954,3.640703,0.480973,0.624384
40,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.581048,0.518087,0.461492,0.502001,0.557051,0.556757,0.504228,0.616687,0.154609,0.001727,8,0.175016,1.334081,3.643307,0.504228,0.556757
59,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.577689,0.557173,0.484597,0.539717,0.573310,0.605405,0.505738,0.540948,0.047665,0.640966,3,0.061225,1.357648,3.681225,0.505738,0.540948
8,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.428137,0.377547,0.557913,0.433190,0.592253,0.612058,0.455301,0.575123,0.187572,0.052213,1,0.258847,1.352152,3.704418,0.455301,0.575123
13,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.553424,0.569231,0.465871,0.523399,0.582868,0.611227,0.498037,0.529249,0.184135,0.079676,5,0.104995,1.337530,3.724049,0.498037,0.529249
6,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.556713,0.610603,0.461870,0.574046,0.567682,0.602079,0.467834,0.558805,0.198094,0.046071,5,0.079883,1.621132,3.750319,0.467834,0.558805
14,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.544938,0.552183,0.479236,0.494458,0.600027,0.601663,0.432045,0.577278,0.171240,0.044248,5,0.102017,1.357341,3.770341,0.432045,0.577278
29,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.657123,0.510187,0.573694,0.544951,0.598821,0.584200,0.472818,0.549261,0.163459,0.001381,9,0.170998,1.307774,3.779985,0.472818,0.549261
17,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.422238,0.466528,0.577167,0.463362,0.595552,0.598753,0.458623,0.543719,0.098710,0.118211,7,0.128904,1.356410,3.798669,0.458623,0.543719


In [ ]:
print(bdd_sor[bdd_sor.min_best_perf >= 0.5].shape[0])
bdd_sor[bdd_sor.min_best_perf >= 0.5]

5


,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,crit,min_best_perf,min_best_intra
30,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.451216,0.523493,0.518121,0.470443,0.539439,0.604158,0.560254,0.536022,0.035590,0.085306,1,0.036188,1.941209,3.609206,0.536022,0.536022
40,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.581048,0.518087,0.461492,0.502001,0.557051,0.556757,0.504228,0.616687,0.154609,0.001727,8,0.175016,1.334081,3.643307,0.504228,0.556757
59,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.577689,0.557173,0.484597,0.539717,0.573310,0.605405,0.505738,0.540948,0.047665,0.640966,3,0.061225,1.357648,3.681225,0.505738,0.540948
58,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.614237,0.488565,0.444428,0.454280,0.550616,0.538046,0.517970,0.511084,0.071475,0.004042,9,0.155039,1.328974,3.849317,0.511084,0.511084
28,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.637555,0.523909,0.461190,0.558190,0.605378,0.557588,0.500151,0.519397,0.218135,0.001597,5,0.151247,1.335749,3.873971,0.500151,0.519397


In [8]:
print(bdd_sor[bdd_sor.min_best_intra >= 0.55].shape[0])
bdd_sor[bdd_sor.min_best_intra >= 0.55]

7


,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,best_crit,min_best_perf,min_best_intra
35,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.904493,0.498545,0.506342,0.667796,0.857813,0.633680,0.480973,0.624384,0.010431,0.003729,3,0.007851,1.079954,3.640703,0.480973,0.624384
40,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.581048,0.518087,0.461492,0.502001,0.557051,0.556757,0.504228,0.616687,0.154609,0.001727,8,0.175016,1.334081,3.643307,0.504228,0.556757
8,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.428137,0.377547,0.557913,0.433190,0.592253,0.612058,0.455301,0.575123,0.187572,0.052213,1,0.258847,1.352152,3.704418,0.455301,0.575123
6,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.556713,0.610603,0.461870,0.574046,0.567682,0.602079,0.467834,0.558805,0.198094,0.046071,5,0.079883,1.621132,3.750319,0.467834,0.558805
14,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.544938,0.552183,0.479236,0.494458,0.600027,0.601663,0.432045,0.577278,0.171240,0.044248,5,0.102017,1.357341,3.770341,0.432045,0.577278
47,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.527983,0.480665,0.484144,0.507543,0.578702,0.602079,0.437330,0.565271,0.395445,0.076689,3,0.187084,5.195068,3.811416,0.437330,0.565271
36,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.500000,0.500000,0.500000,0.500000,0.579160,0.604990,0.446542,0.553264,0.358928,0.496460,3,0.008309,1440.647949,3.816434,0.446542,0.553264


# S.T.s.

In [9]:
bdd_sts = pd.read_csv("/neurospin/dico/agaudin/Runs/09_new_repo/Output/grid_searches/step2/S.T.s./bdd_STs_both.csv", index_col=0)

bdd_sts['best_crit'] = gs_crit_full(bdd_sts)
bdd_sts['min_best_perf'] = bdd_sts[['best_train_auc', 'best_val_auc', 'best_test_intra_auc', 'best_test_auc']].apply(min, axis=1)
bdd_sts['min_best_intra'] = bdd_sts[['best_train_auc', 'best_val_auc', 'best_test_intra_auc']].apply(min, axis=1)

# sort by criterion
bdd_sts.sort_values(by='best_crit', inplace=True)

bdd_sts

,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,best_crit,min_best_perf,min_best_intra
7,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.719751,0.515593,0.542585,0.744766,0.617458,0.599168,0.582301,0.668719,0.164393,0.000475,5,0.031012,1.258531e+00,3.169413,0.582301,0.599168
22,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.849605,0.518503,0.531108,0.654557,0.641476,0.606237,0.542736,0.702278,0.005153,0.002315,4,0.023665,9.968667e-01,3.209326,0.542736,0.606237
2,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.992891,0.472765,0.546814,0.696121,0.682077,0.576299,0.528994,0.766933,0.131382,0.000258,1,0.167687,8.858657e-01,3.235108,0.528994,0.576299
11,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.756400,0.537214,0.525974,0.718904,0.609911,0.598753,0.579432,0.582512,0.036740,0.000459,5,0.062965,1.231797e+00,3.327819,0.579432,0.582512
3,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.918962,0.491892,0.511326,0.652401,0.598580,0.570478,0.568408,0.613916,0.059643,0.000869,3,0.168723,1.055657e+00,3.370844,0.568408,0.570478
18,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.913106,0.501871,0.511477,0.521552,0.576105,0.583784,0.578979,0.579126,0.026228,0.002995,1,0.421370,1.060900e+00,3.377588,0.576105,0.576105
20,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.599778,0.610395,0.588946,0.562808,0.582046,0.620374,0.590607,0.542796,0.027290,0.008931,4,0.384765,1.359767e+00,3.414495,0.542796,0.542796
39,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.610821,0.511850,0.582301,0.701663,0.554116,0.589189,0.586228,0.596983,0.192482,0.004856,0,0.029177,1.344788e+00,3.457022,0.554116,0.554116
6,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.724171,0.521830,0.504077,0.728140,0.713138,0.528067,0.507098,0.713978,0.109223,0.000318,5,0.473691,1.278136e+00,3.467392,0.507098,0.528067
47,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.645805,0.518919,0.533525,0.762315,0.585041,0.587526,0.493506,0.701355,0.160540,0.007255,2,0.363501,1.296191e+00,3.475476,0.493506,0.585041


In [10]:
print(bdd_sts[bdd_sts.min_best_perf >= 0.5].shape[0])
bdd_sts[bdd_sts.min_best_perf >= 0.5]

37


,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,best_crit,min_best_perf,min_best_intra
7,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.719751,0.515593,0.542585,0.744766,0.617458,0.599168,0.582301,0.668719,0.164393,0.000475,5,0.031012,1.258531e+00,3.169413,0.582301,0.599168
22,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.849605,0.518503,0.531108,0.654557,0.641476,0.606237,0.542736,0.702278,0.005153,0.002315,4,0.023665,9.968667e-01,3.209326,0.542736,0.606237
2,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.992891,0.472765,0.546814,0.696121,0.682077,0.576299,0.528994,0.766933,0.131382,0.000258,1,0.167687,8.858657e-01,3.235108,0.528994,0.576299
11,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.756400,0.537214,0.525974,0.718904,0.609911,0.598753,0.579432,0.582512,0.036740,0.000459,5,0.062965,1.231797e+00,3.327819,0.579432,0.582512
3,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.918962,0.491892,0.511326,0.652401,0.598580,0.570478,0.568408,0.613916,0.059643,0.000869,3,0.168723,1.055657e+00,3.370844,0.568408,0.570478
18,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.913106,0.501871,0.511477,0.521552,0.576105,0.583784,0.578979,0.579126,0.026228,0.002995,1,0.421370,1.060900e+00,3.377588,0.576105,0.576105
20,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.599778,0.610395,0.588946,0.562808,0.582046,0.620374,0.590607,0.542796,0.027290,0.008931,4,0.384765,1.359767e+00,3.414495,0.542796,0.542796
39,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.610821,0.511850,0.582301,0.701663,0.554116,0.589189,0.586228,0.596983,0.192482,0.004856,0,0.029177,1.344788e+00,3.457022,0.554116,0.554116
6,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.724171,0.521830,0.504077,0.728140,0.713138,0.528067,0.507098,0.713978,0.109223,0.000318,5,0.473691,1.278136e+00,3.467392,0.507098,0.528067
37,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.566634,0.511435,0.553760,0.588054,0.550280,0.611642,0.590003,0.555727,0.141279,0.027113,2,0.048906,1.347586e+00,3.491227,0.550280,0.550280


In [11]:
print(bdd_sts[bdd_sts.min_best_intra >= 0.55].shape[0])
bdd_sts[bdd_sts.min_best_intra >= 0.55]

13


,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,best_crit,min_best_perf,min_best_intra
7,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.719751,0.515593,0.542585,0.744766,0.617458,0.599168,0.582301,0.668719,0.164393,0.000475,5,0.031012,1.258531,3.169413,0.582301,0.599168
22,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.849605,0.518503,0.531108,0.654557,0.641476,0.606237,0.542736,0.702278,0.005153,0.002315,4,0.023665,0.996867,3.209326,0.542736,0.606237
2,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.992891,0.472765,0.546814,0.696121,0.682077,0.576299,0.528994,0.766933,0.131382,0.000258,1,0.167687,0.885866,3.235108,0.528994,0.576299
11,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.756400,0.537214,0.525974,0.718904,0.609911,0.598753,0.579432,0.582512,0.036740,0.000459,5,0.062965,1.231797,3.327819,0.579432,0.582512
3,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.918962,0.491892,0.511326,0.652401,0.598580,0.570478,0.568408,0.613916,0.059643,0.000869,3,0.168723,1.055657,3.370844,0.568408,0.570478
18,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.913106,0.501871,0.511477,0.521552,0.576105,0.583784,0.578979,0.579126,0.026228,0.002995,1,0.421370,1.060900,3.377588,0.576105,0.576105
39,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.610821,0.511850,0.582301,0.701663,0.554116,0.589189,0.586228,0.596983,0.192482,0.004856,0,0.029177,1.344788,3.457022,0.554116,0.554116
47,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.645805,0.518919,0.533525,0.762315,0.585041,0.587526,0.493506,0.701355,0.160540,0.007255,2,0.363501,1.296191,3.475476,0.493506,0.585041
10,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.613891,0.546778,0.483540,0.642857,0.652167,0.594595,0.471761,0.658251,0.010350,0.002817,4,0.012746,1.344729,3.490516,0.471761,0.594595
37,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.566634,0.511435,0.553760,0.588054,0.550280,0.611642,0.590003,0.555727,0.141279,0.027113,2,0.048906,1.347586,3.491227,0.550280,0.550280


# pericalcarine

In [12]:
bdd_pc = pd.read_csv("/neurospin/dico/agaudin/Runs/09_new_repo/Output/grid_searches/step2/pericalcarine/bdd_pericalcarine_both.csv", index_col=0)

bdd_pc['best_crit'] = gs_crit_full(bdd_pc)
bdd_pc['min_best_perf'] = bdd_pc[['best_train_auc', 'best_val_auc', 'best_test_intra_auc', 'best_test_auc']].apply(min, axis=1)
bdd_pc['min_best_intra'] = bdd_pc[['best_train_auc', 'best_val_auc', 'best_test_intra_auc']].apply(min, axis=1)

# sort by criterion
bdd_pc.sort_values(by='best_crit', inplace=True)

bdd_pc

,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,best_crit,min_best_perf,min_best_intra
0,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.704169,0.607484,0.475989,0.648091,0.619461,0.624116,0.544397,0.640394,0.296929,0.001980,10,0.204315,1.270310,3.243913,0.544397,0.619461
10,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.841438,0.625364,0.501812,0.575739,0.632587,0.637422,0.518121,0.641933,0.046062,0.001222,10,0.059264,1.203747,3.268520,0.518121,0.632587
1,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.673136,0.584615,0.544095,0.629926,0.603920,0.613306,0.541981,0.654249,0.078573,0.002264,5,0.040576,1.307342,3.294743,0.541981,0.603920
5,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.940289,0.505613,0.508910,0.551416,0.595336,0.659044,0.538055,0.625000,0.069306,0.001646,6,0.018243,0.942862,3.315782,0.538055,0.595336
9,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.847394,0.647817,0.463304,0.616687,0.772302,0.661538,0.535035,0.609914,0.007678,0.002097,7,0.008789,1.174820,3.352842,0.535035,0.609914
12,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.925048,0.585863,0.508306,0.654249,0.582151,0.661123,0.530504,0.625616,0.037515,0.001438,8,0.010468,1.000327,3.375295,0.530504,0.582151
7,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.730731,0.554262,0.541377,0.518781,0.580697,0.653638,0.515101,0.664409,0.070430,0.002952,4,0.006194,1.257750,3.394558,0.515101,0.580697
13,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.713479,0.612058,0.565539,0.596367,0.559269,0.636590,0.538357,0.617303,0.007401,0.004491,10,0.147132,1.301262,3.453227,0.538357,0.559269
2,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.500000,0.500000,0.500000,0.500000,0.531221,0.623701,0.531561,0.696275,0.153283,0.400237,9,0.012693,17133.101562,3.492356,0.531221,0.531221
6,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.549451,0.611850,0.551948,0.556188,0.551055,0.604990,0.572335,0.568042,0.102405,0.034395,4,0.365190,1.359043,3.499360,0.551055,0.551055
